In [ ]:
# Import Pyomo
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

Optimize Delivery Routes for a Local
Delivery Company

In [ ]:
# Inputs
trucks = ['T1','T2','T3','T4','T5']
customers = ['C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','C15']

In [ ]:
# Create model
model = ConcreteModel()

# Parameters
distance = {
    ('T1','C1'): 4, ('T1','C2'): 6, ('T1','C3'): 5,
    ('T2','C1'): 5, ('T2','C2'): 4, ('T2','C3'): 7,
}

# Max customers per truck
capacity = {'T1': 5, 'T2': 5, 'T3': 5, 'T4': 5, 'T5': 5}

# Max demand from customer
demand = {'C1':1, 'C2':1, 'C3':1, 'C4':1, 'C5':1, 'C6':1, 'C7':1, 'C8':1, 'C9':1, 'C10':1,
          'C11':1, 'C12':1, 'C13':1, 'C14':1, 'C15':1}

# Decision variables
model.x = Var(trucks, customers, domain=Binary)

# Objective: minimize total distance
def total_distance(model):
    return sum(model.x[t,c]*distance.get((t,c), 10) for t in trucks for c in customers)
model.obj = Objective(rule=total_distance, sense=minimize)

# Constraints

# Each customer visited exactly once
def customer_visit_rule(model, c):
    return sum(model.x[t,c] for t in trucks) == 1
model.customer_visit = Constraint(customers, rule=customer_visit_rule)

# Truck capacity constraints
def truck_capacity_rule(model, t):
    return sum(model.x[t,c]*demand[c] for c in customers) <= capacity[t]
model.truck_capacity = Constraint(trucks, rule=truck_capacity_rule)

In [ ]:
# Solve the model
!apt-get install -y -qq coinor-cbc
import shutil
cbc_path = shutil.which('cbc')
if cbc_path is None:
    raise RuntimeError("CBC solver not found even after installation. Please check installation.")
solver = SolverFactory('cbc', executable=cbc_path)
results = solver.solve(model, tee=True)

Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3) ...
Selecting previously unselected package coinor-libcbc3:amd64.
Preparing to unpack .../4-coinor-libcbc3_2.10.7+ds1-1_amd64.deb ...
Unpacking coinor-libcbc3:

In [ ]:
# Display results
for t in trucks:
    assigned = [c for c in customers if value(model.x[t,c]) > 0.5]
    print(f"Truck {t} delivers to: {assigned}")

Truck T1 delivers to: ['C1', 'C3', 'C8', 'C14', 'C15']
Truck T2 delivers to: ['C2', 'C9']
Truck T3 delivers to: ['C4', 'C5', 'C12', 'C13']
Truck T4 delivers to: ['C7', 'C10']
Truck T5 delivers to: ['C6', 'C11']
